# Census API - New York

In [1]:
# import packages

from us import states
import pandas as pd
import requests

# import doc with API key

import config

#### Census API Link References

In [ ]:
# "https://api.census.gov/data/2019/acs/acs5/profile?get=NAME,DP05_0001E&for=state:36&key={0}".format(config.MY_API_KEY)
# "https://api.census.gov/data/2019/acs/acs5/profile?get=group(DP05)&for=school%20district%20(unified):29850&in=state:36&key={0}".format(config.MY_API_KEY)

variableURL = "https://api.census.gov/data/2019/acs/acs5/profile/variables"
censusVarNames = requests.request("GET", variableURL)
# print(censusVarNames.text)

## Import CSV

#### If a user is interested in looking at only select parts of the entire US Census set, save a separate text file with your list. 
#### You can pick and choose which school districts after compiling all the data, but it helps to filter out the data earlier on to minimize the size of your data frame

In [33]:
# This list is the top 20 school districts in Nassau County.

selectedNYCounties = pd.read_csv(r"Data\USCensusNYSchDist.txt",names=['School District Name'])
selectedNYCounties.head()

,School District Name
0,Jericho Union Free School District
1,Great Neck Public Schools
2,Syosset Central School District
3,Roslyn Union Free School District
4,Herricks Union Free School District


In [15]:
# Read Census data set which lists GEOIDs per each school district in state

URL = ("https://www2.census.gov/geo/docs/reference/codes/files/st36_ny_schdist.txt")
CountyGEOID = pd.read_csv(URL,names=['State', 'ID','GEOID','School District Name', 'Class'])
CountyGEOID.head()

,State,ID,GEOID,School District Name,Class
0,NY,36,1,Dolgeville Central School District,Unified
1,NY,36,2,Sauquoit Valley Central School District,Unified
2,NY,36,3,Edwards-Knox Central School District,Unified
3,NY,36,4,Rotterdam-Mohonasen Central School District,Unified
4,NY,36,5,Broadalbin-Perth Central School District,Unified


#### GEOIDs are going to be critical to using the Census API.

Create a new Dataframe, which combines all rows based on their "School District Name."
This will allow that first layer of filtering to happen -> you get the GEOIDs for all of the school districts you are interested in looking at.

Need to further investigate and understand Pandas concat, merge, join. Not sure why merge worked - so need to dig into that.


In [14]:
NYCountiesGEOID = NYCounties.merge(data, on=['School District Name'],how='inner')
NYCountiesGEOID

,School District Name,State,ID,GEOID,Class
0,Jericho Union Free School District,NY,36,15810,Unified
1,Syosset Central School District,NY,36,28560,Unified
2,Roslyn Union Free School District,NY,36,25050,Unified
3,Herricks Union Free School District,NY,36,14280,Unified
4,Manhasset Union Free School District,NY,36,18270,Unified
5,East Williston Union Free School District,NY,36,10050,Unified
6,North Shore Central School District,NY,36,26370,Unified
7,Hewlett-Woodmere Union Free School District,NY,36,31710,Unified
8,Plainview-Old Bethpage Central School District,NY,36,23220,Unified
9,Bellmore-Merrick Central High School District,NY,36,19020,Secondary


In [16]:
# Get all GEO IDs in a single list

GEOIDonly = joined['GEOID'].tolist()
separator = ', '

x = separator.join(map(str, GEOIDonly))
print(x)

15810, 28560, 25050, 14280, 18270, 10050, 26370, 31710, 23220, 19020, 11760, 23580, 4740, 17910, 17700, 24780, 18630, 29850, 17160


In [17]:
# census variables

cenVar = 'B01001_001E'

In [22]:
def jsontodf(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [20]:
def schoolDistrictInput(variables,strings):
    URL = "https://api.census.gov/data/2019/acs/acs5?get=NAME,{0}&for=school%20district%20(unified):{1}&in=state:36&key={2}".format(variables,strings,config.MY_API_KEY)
    return requests.request("GET", URL)

In [24]:
test = schoolDistrictInput(cenVar, x)
jsontodf(test).head()

,NAME,B01001_001E,state,school district (unified)
0,"Roslyn Union Free School District, New York",18598,36,25050
1,"East Williston Union Free School District, New...",8886,36,10050
2,"North Shore Central School District, New York",16648,36,26370
3,"Garden City Union Free School District, New York",22499,36,11760
4,"Wantagh Union Free School District, New York",16739,36,29850
